# Llama.cpp

>[llama.cpp python](https://github.com/abetlen/llama-cpp-python) 库是 `@ggerganov`
>[llama.cpp](https://github.com/ggerganov/llama.cpp) 的简单 Python 绑定。
>
>该包提供：
>
> - 通过 ctypes 接口对 C API 的低级访问。
> - 用于文本补全的高级 Python API
>   - 类似 `OpenAI` 的 API
>   - `LangChain` 兼容性
>   - `LlamaIndex` 兼容性
> - 类似 OpenAI 的 Web 服务器
>   - 本地 Copilot 替代品
>   - 函数调用支持
>   - 视觉 API 支持
>   - 多模型

## 概述

### 集成详情
| 类 | 包 | 本地 | 可序列化 | JS 支持 |
| :--- | :--- | :---: | :---: |  :---: |
| [ChatLlamaCpp](https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.llamacpp.ChatLlamaCpp.html) | [langchain-community](https://python.langchain.com/api_reference/community/index.html) | ✅ | ❌ | ❌ |

### 模型特性
| [工具调用](/docs/how_to/tool_calling) | [结构化输出](/docs/how_to/structured_output/) | JSON 模式 | 图像输入 | 音频输入 | 视频输入 | [Token级流式输出](/docs/how_to/chat_streaming/) | 原生异步 | [Token 用量跟踪](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ❌ | ❌ | ❌ | ❌ | ✅ | ❌ | ❌ | ✅ | 

## 设置

为了开始并使用下面展示的**所有**功能，我们推荐使用经过工具调用微调的模型。

我们将使用 NousResearch 的 [Hermes-2-Pro-Llama-3-8B-GGUF](https://huggingface.co/NousResearch/Hermes-2-Pro-Llama-3-8B-GGUF)。

> Hermes 2 Pro 是 Nous Hermes 2 的升级版本，它包含一个更新和清理过的 OpenHermes 2.5 数据集，以及一个内部开发的全新函数调用和 JSON 模式数据集。新版本的 Hermes 在保持其出色的通用任务和对话能力的同时，在函数调用方面也表现出色。

请参阅我们关于本地模型的指南以了解更多详情：

* [在本地运行 LLM](https://python.langchain.com/v0.1/docs/guides/development/local_llms/)
* [将本地模型与 RAG 结合使用](https://python.langchain.com/v0.1/docs/use_cases/question_answering/local_retrieval_qa/)

### 安装

LangChain LlamaCpp 集成位于 `langchain-community` 和 `llama-cpp-python` 包中：

In [ ]:
%pip install -qU langchain-community llama-cpp-python

## 实例化

现在我们可以实例化我们的模型对象并生成聊天补全：

In [14]:
# Path to your model weights
local_model = "local/path/to/Hermes-2-Pro-Llama-3-8B-Q8_0.gguf"

In [ ]:
import multiprocessing

from langchain_community.chat_models import ChatLlamaCpp

llm = ChatLlamaCpp(
    temperature=0.5,
    model_path=local_model,
    n_ctx=10000,
    n_gpu_layers=8,
    n_batch=300,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    max_tokens=512,
    n_threads=multiprocessing.cpu_count() - 1,
    repeat_penalty=1.5,
    top_p=0.5,
    verbose=True,
)

## 调用

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

ai_msg = llm.invoke(messages)
ai_msg

In [4]:
print(ai_msg.content)

J'aime programmer. (In France, "programming" is often used in its original sense of scheduling or organizing events.) 

If you meant computer-programming: 
Je suis amoureux de la programmation informatique.

(You might also say simply 'programmation', which would be understood as both meanings - depending on context).


## 链式调用

我们可以像这样将我们的模型与提示模板进行[链式调用](/docs/how_to/sequence/)：

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

## 工具调用

首先，它的工作方式与 OpenAI 的函数调用（Function Calling）大体相同。

OpenAI 拥有一个 [工具调用](https://platform.openai.com/docs/guides/function-calling)（我们在此互换使用“工具调用”和“函数调用”）API，它允许你描述工具及其参数，并让模型返回一个 JSON 对象，其中包含要调用的工具和该工具的输入。工具调用对于构建使用工具的链和代理，以及更广泛地从模型获取结构化输出极其有用。

通过 `ChatLlamaCpp.bind_tools`，我们可以轻松地将 Pydantic 类、字典模式、LangChain 工具，甚至函数作为工具传递给模型。在底层，它们会被转换为 OpenAI 工具模式，如下所示：
```
{
    "name": "...",
    "description": "...",
    "parameters": {...}  # JSONSchema
}
```
并在每次模型调用中传递。

但是，它不能自动触发函数/工具，我们需要通过指定 'tool choice' 参数来强制它。此参数通常格式如下。

```{"type": "function", "function": {"name": <<tool_name>>}}.```

In [19]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field


class WeatherInput(BaseModel):
    location: str = Field(description="The city and state, e.g. San Francisco, CA")
    unit: str = Field(enum=["celsius", "fahrenheit"])


@tool("get_current_weather", args_schema=WeatherInput)
def get_weather(location: str, unit: str):
    """Get the current weather in a given location"""
    return f"Now the weather in {location} is 22 {unit}"


llm_with_tools = llm.bind_tools(
    tools=[get_weather],
    tool_choice={"type": "function", "function": {"name": "get_current_weather"}},
)

In [ ]:
ai_msg = llm_with_tools.invoke(
    "what is the weather like in HCMC in celsius",
)

In [21]:
ai_msg.tool_calls

[{'name': 'get_current_weather',
  'args': {'location': 'Ho Chi Minh City', 'unit': 'celsius'},
  'id': 'call__0_get_current_weather_cmpl-394d9943-0a1f-425b-8139-d2826c1431f2'}]

In [ ]:
class MagicFunctionInput(BaseModel):
    magic_function_input: int = Field(description="The input value for magic function")


@tool("get_magic_function", args_schema=MagicFunctionInput)
def magic_function(magic_function_input: int):
    """Get the value of magic function for an input."""
    return magic_function_input + 2


llm_with_tools = llm.bind_tools(
    tools=[magic_function],
    tool_choice={"type": "function", "function": {"name": "get_magic_function"}},
)

ai_msg = llm_with_tools.invoke(
    "What is magic function of 3?",
)

ai_msg

In [26]:
ai_msg.tool_calls

[{'name': 'get_magic_function',
  'args': {'magic_function_input': 3},
  'id': 'call__0_get_magic_function_cmpl-cd83a994-b820-4428-957c-48076c68335a'}]

# 结构化输出

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from pydantic import BaseModel


class Joke(BaseModel):
    """A setup to a joke and the punchline."""

    setup: str
    punchline: str


dict_schema = convert_to_openai_tool(Joke)
structured_llm = llm.with_structured_output(dict_schema)
result = structured_llm.invoke("Tell me a joke about birds")
result

In [27]:
result

{'setup': '- Why did the chicken cross the playground?',
 'punchline': '\n\n- To get to its gilded cage on the other side!'}

# 流式处理

In [ ]:
for chunk in llm.stream("what is 25x5"):
    print(chunk.content, end="\n", flush=True)

## API 参考

如需了解 ChatLlamaCpp 所有功能和配置的详细文档，请访问 API 参考：https://python.langchain.com/api_reference/community/chat_models/langchain_community.chat_models.llamacpp.ChatLlamaCpp.html